# Skull Stripping & MRI Image Shrinking #

In [117]:
import numpy as np
import os
import pathlib
import warnings
from nilearn import image
from nilearn import plotting
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import fnmatch

## Stripping MRI Images
#### Here we extract stripped MRI images using grey matter, white matter, Cerebrospinal fluid (CSF)

In [118]:
def brainextr(name, name_c1, name_c2, name_c3):

    img = image.load_img(name).get_data()[:,:,:]
    img_b = (image.load_img(name_c1).get_data()[:,:,:] + 
             image.load_img(name_c2).get_data()[:,:,:] + 
             image.load_img(name_c3).get_data()[:,:,:] )
            
    img_ind = img_b > 0.01
    imgf = img * img_ind
    imgf = ShrinkImg(imgf)
    
    
    return imgf

## Shrinking MRI Images
#### Here we remove planes in the 3D image that do not contain any tissue 

In [119]:
def ShrinkImg(Img):

    Imgx = np.sum(Img, axis=1)
    Imgx = np.sum(Imgx, axis=1)
    x = np.nonzero(Imgx)

    minx = x[0][0]
    maxx = x[0][-1]

    Imgy = np.sum(Img, axis=0)
    Imgy = np.sum(Imgy, axis=1)
    y = np.nonzero(Imgy)

    miny = y[0][0]
    maxy = y[0][-1]

    Imgz = np.sum(Img, axis=0)
    Imgz = np.sum(Imgz, axis=0)
    z = np.nonzero(Imgz)

    minz = z[0][0]
    maxz = z[0][-1]

    Imgf = Img[minx : maxx + 1, miny : maxy + 1, minz : maxz + 1]

    maxdx = maxx - minx + 1
    maxdy = maxy - miny + 1
    maxdz = maxz - minz + 1
    
    return Imgf, maxdx, maxdy, maxdz

## Extracting ID of MRI Images
#### Here we gather the ID of MRI images into a set called set_imageData

In [120]:
rootPath = './All_Stripped/'                                  # directory for stripped MRI images 
pattern = '*.nii'
                                     
dataFiles = []
set_imageData = set()

for root, dirs, files in os.walk(rootPath):
    for filename in fnmatch.filter(files, pattern):
        dataFiles = (os.path.join(root, filename)) 
        k = str((dataFiles.split("ADNI_"))[1]  )               # subject_ID
        k = (k.split("_MR_"))
        key = k[0]
        set_imageData.update([key])

#### Here we extract the corresponding grey matter, white matter, Cerebrospinal fluid (CSF) data for each MRI image ID and pass them to brainextr function

In [127]:
mypath = './All_Stripped/'
counter = 0
dict_c1 = dict()
dict_c2 = dict()
dict_c3 = dict()
dict_img = dict()
for key in set_imageData:
    pattern = str(key)
    if not os.path.isfile('./numpyArrays/' + pattern + '.npy'):
        for file in os.listdir(mypath):
            if fnmatch.fnmatch(file, "c1" + "*" + pattern + "*"):
                dict_c1[pattern] = file
            elif fnmatch.fnmatch(file, "c2" + "*" + pattern + "*"):
                dict_c2[pattern] = file
            elif fnmatch.fnmatch(file, "c3" + "*" + pattern + "*"):
                dict_c3[pattern] = file
            elif fnmatch.fnmatch(file, "*" + pattern + "*"):
                dict_img[pattern] = file
        
max_x = max_y = max_z = 0

for img_ID in set_imageData:
    pattern = str(img_ID)
    if not os.path.isfile('./numpyArrays/' + pattern + '.npy'):
        name = os.path.join(root, str(dict_img[pattern]))
        name_c1 = os.path.join(root, str(dict_c1[pattern]))
        name_c2 = os.path.join(root, str(dict_c2[pattern]))
        name_c3 = os.path.join(root, str(dict_c3[pattern]))
        imgf, maxx, maxy, maxz = brainextr(name, name_c1, name_c2, name_c3 )
        np.save('./numpyArrays/' + pattern + '.npy', imgf)
        if(maxx > max_x):
            max_x = maxx
        if(maxy > max_y):
            max_y = maxy
        if(maxz > max_z):
            max_z = maxz

        print('image ID: ', pattern, ', image size:', imgf.shape, ', [max_x, max_y, max_z] = ', max_x, max_y, max_z)

    f = open('./numpyArrays/'  + "maxANDmin.txt", "a");
    f.write("[max_x, max_y, max_z] = " + str(max_x) + ", " + str(max_y) + ", " + str(max_z) + '\n')
    f.close()


image ID:  009_S_1334 , image size: (179, 189, 120) , [max_x, max_y, max_z] =  179 189 120
image ID:  009_S_1199 , image size: (167, 188, 126) , [max_x, max_y, max_z] =  179 189 126
image ID:  009_S_1354 , image size: (148, 180, 109) , [max_x, max_y, max_z] =  179 189 126
